<a href="https://colab.research.google.com/github/IlyaKo4atkov/IlyaKo4atkov/blob/main/the_prognosis_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Датасет о стоимости акций Сбербанка с 01.01.2013 года:

https://storage.yandexcloud.net/academy.ai/SBER.csv

Визуализация индикатора "Полосы Боллинджера", проанализировал график, и предложил вариант торговли акциями Сбербанка с помощью этого инструмента.


* линия посередине — это простая скользящая средняя (SMA) с периодом `ma_size`, обычно около 20 дней;

* верхняя и нижняя линии (BB) — построены на основе SMА, но к нему добавлено стреднеквадратичное отклонение:

```
 SMA = data['close'].rolling(ma_size).mean()
 BB_UP = SMA + data['close'].rolling(ma_size).std() * bol_size
 BB_DOWN = SMA - data['close'].rolling(ma_size).std() * bol_size
   
```

где bol_size - ширина коридора, подбирается по графику. Выберите такое его значение, чтобы по графику можно было принимать торговые решения.

In [ ]:

# Загрузим все необходимые библиотеки

import os
import numpy as np
import pandas as pd

import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

url = "https://storage.yandexcloud.net/academy.ai/SBER.csv"
df = pd.read_csv('SBER.csv', delimiter=';')
#Функция преобразования даты в формальный вид ГГГГ-ММ-ДД
def sep_date(x):
  x = str(x)
  new_date = x[:4] + '-' + x[4:6] + '-' + x[6:]
  return new_date
df['DATE'] = df['DATE'].apply(sep_date)
df['DATE'] = pd.to_datetime(df['DATE'])

# SMA - расчет скользящих средних
SMA = df['CLOSE'].rolling(20).mean()
df['SMA_20'] = df['CLOSE'].rolling(20).mean()
#Так как коридор, это диапазон цен, в котором актив торгуется в определенный момент времени, за кородор был выбран период с мая 2015 по октябрь 2015
#этот диапазон имеет стабильные границы максимальной и минимальной цены актива
bol_size = 150 # примерное количество дней за 5 месяцев
df['BB_UP'] = SMA + df['CLOSE'].rolling(20).std()
df['BB_DOWN'] = SMA - df['CLOSE'].rolling(20).std()
fig = go.Figure()
fig.add_scatter(x=df.DATE, y=df.SMA_20, name='SMA 20')
fig.add_scatter(x=df.DATE, y=df.BB_UP, name='BB_UP')
fig.add_scatter(x=df.DATE, y=df.BB_DOWN, name='BB_DOWN')
fig.add_scatter(x=df.DATE, y=df.CLOSE, name='CLOSE', opacity=0.3)
fig.update_xaxes(rangeslider_visible=True)
fig.show()


In [ ]:
#Построение временного ряда. Для более удобное визуальзации и нахождения оптимального коридора, был построен OHLC график
fig = go.Figure(data=[go.Candlestick(x=df['DATE'],
                open=df['OPEN'],
                high=df['HIGH'],
                low=df['LOW'],
                close=df['CLOSE'])])
fig.update_xaxes(rangeslider_visible=True)
fig.show()

Вывод:
Таким образом восходящая тенденция и узкий торговый коридор могут указывать на возможность покупки актива в ожидании его роста. Если же рассматривать нисходящую тенденцию и широкий торговый коридор, то это может указывать на возможность продаж актива в ожидании его падения.